In [3]:
import yfinance as yf
import trader
import numpy as np

In [9]:
import os
import pandas as pd

# pair_infos = ['BZQ_ERY_2017', 'CATH_SPLG_2018', 'HIPS_TPYP_2019', 'LABD_REW_2020', 'CATH_SPLG_2021']
pair_infos = ['FUTY_VPU_2017', 'VEU_VXUS_2018', 'IVW_VOOG_2019', 'FENY_VDE_2020', 'SPYG_VOOG_2021' ]
# pair_infos = ['BRZU_DUST_2022',]


path_output = "./outputs/outputs_PCA/"
graph_output = False

In [3]:
# pair = ["BZQ", "ERY"]
# prices = yf.download(pair,start="2015-01-01",end="2016-12-31")["Adj Close"]
# prices_present, signals = trader.generate_signals_dynamic(prices, threshold=0.2, freq=1, rolling=True)


In [4]:
# df_return = trader.calculate_return(prices_present,0,signals,"cumulative1")

In [5]:
# from matplotlib import pyplot as plt
# # plt.plot((df_return["unrealized_pnl_cum"].pct_change()+1).fillna(0).replace([np.Inf, -np.inf],1))
# plt.plot(df_return["unrealized_pnl_cum"])
# plt.plot(df_return["unrealized_pnl_cum"].expanding().std().fillna(0))
# df_return["unrealized_pnl_cum"].expanding().std().fillna(0)

In [15]:
ret_summary = {}

for pair_info in pair_infos:
    print("-------------------------------------------------")
    print("running", pair_info)
    # try:
    x,y,year = pair_info.split("_")
    year = int(year)
    prices = yf.download([x,y],start=str(year-2)+"-01-01",end=str(year)+"-12-31")["Adj Close"]

    prices_present, signals = trader.generate_signals_dynamic(prices, threshold=0.49, freq=1, rolling=True)

    ret_info = {}
    for strategy in ["cumulative1", "cumulative2"]:
        print("...", strategy)
        df_return = trader.calculate_return(prices_present,0,signals,strategy)
        if graph_output:
            fig_name = pair_info+" "+strategy+" freq1"
            f, _ = trader.plot_summary(df_return, fig_name)
            f.savefig(path_output+fig_name+".png")
        ret_summary[pair_info+" "+strategy] = trader.get_summary_copula(df_return, prices)

    # ret_summary[pair_info] = ret_info
    # except:
    #     print("failed signal generation for",pair_info,"... continue")
    #     pass

# # money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
# # money_ret_summary.columns = df_signals.columns
money_ret_summary.to_excel(path_output+"result_summary_0.49_5y_2022_0227.xlsx")

print("DONE")

-------------------------------------------------
running FUTY_VPU_2017
[*********************100%***********************]  2 of 2 completed
historical 0 503
present 504 754
frank is optimal for FUTY and VPU
optimal copula used:  frank
frank is optimal for FUTY and VPU
... cumulative1
... cumulative2
-------------------------------------------------
running VEU_VXUS_2018
[*********************100%***********************]  2 of 2 completed
historical 0 502
present 503 752
frank is optimal for VEU and VXUS
optimal copula used:  frank
frank is optimal for VEU and VXUS
... cumulative1
... cumulative2
-------------------------------------------------
running IVW_VOOG_2019
[*********************100%***********************]  2 of 2 completed
historical 0 501
present 502 752
frank is optimal for IVW and VOOG
optimal copula used:  frank
frank is optimal for IVW and VOOG
... cumulative1
... cumulative2
-------------------------------------------------
running FENY_VDE_2020
[*********************

In [ ]:
# money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
# money_ret_summary.columns = df_signals.columns
money_ret_summary.to_excel(path_output+"result_summary.xlsx")



In [31]:
# pair_infos = ['BZQ_ERY_2017', 'CATH_SPLG_2018', 'HIPS_TPYP_2019', 'LABD_REW_2020', 'CATH_SPLG_2021']
# pair_infos = ['FUTY_VPU_2017', 'VEU_VXUS_2018', 'IVW_VOOG_2019', 'FENY_VDE_2020', 'SPYG_VOOG_2021' ]
pair_infos = ['BRZU_DUST_2022',]
path_output = "./outputs/outputs_ML/"
ret_summary = {}
pairs_pnl = pd.DataFrame()
pairs_pnl_all = pd.DataFrame()
strategy = "cumulative1"
prev_cum_pnl = 0

for pair_info in pair_infos:
    print("-------------------------------------------------")
    print("running", pair_info)
    # try:
    x,y,year = pair_info.split("_")
    year = int(year)
    prices = yf.download([x,y],start=str(year-2)+"-01-01",end=str(year)+"-12-31")["Adj Close"]

    prices_present, signals = trader.generate_signals_dynamic(prices, threshold=0.2, freq=1, rolling=True)

    ret_info = {}
    df_return = trader.calculate_return(prices_present,0,signals,strategy)
    ret_summary[pair_info+" "+strategy] = trader.get_summary_copula(df_return, prices)

    pairs_pnl = df_return[["unrealized_pnl_cum","unrealized_PnL", "realized_pnl_daily", "realized_max_drawdown", "realized_avg_drawdown"]].fillna(0)
    if pairs_pnl.empty :
        pairs_pnl_all = pairs_pnl.copy()
    else: 
        pairs_pnl["unrealized_pnl_cum"] = df_return["unrealized_pnl_cum"] + prev_cum_pnl
        pairs_pnl_all = pairs_pnl_all.append(pairs_pnl)

    prev_cum_pnl = prev_cum_pnl + df_return["unrealized_pnl_cum"][-1]

    # print(df_return["unrealized_pnl_cum"])
    # ret_summary[pair_info] = ret_info
    # except:
    #     print("failed signal generation for",pair_info,"... continue")
    #     pass

# # money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
# money_ret_summary = pd.DataFrame.from_dict(ret_summary, orient='index')
# # money_ret_summary.columns = df_signals.columns
# money_ret_summary.to_excel(path_output+"result_summary_0.35_1.xlsx")

print("DONE")

pairs_pnl_all["unrealized_max_drawdown"] = ((pairs_pnl_all["unrealized_pnl_cum"].cummax() - pairs_pnl_all["unrealized_pnl_cum"])/pairs_pnl_all["unrealized_pnl_cum"].cummax()).fillna(0).replace([np.inf,-np.inf],0)
pairs_pnl_all["unrealized_avg_drawdown"] = pairs_pnl_all['unrealized_max_drawdown'].expanding().mean().fillna(0)



print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_2022_cum1_226.xlsx")

-------------------------------------------------
running BRZU_DUST_2022
[*********************100%***********************]  2 of 2 completed
historical 0 504
present 505 755
gumbel is optimal for BRZU and DUST
optimal copula used:  gumbel
gumbel is optimal for BRZU and DUST
DONE
0.13219009891657177
0.06383528962340511
0.0
0.0


In [7]:
# ML 5y cum1

print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_5y_cum1_226.xlsx")

0.5194360762879934
0.07501257082775757
6.695559685012056
0.5698348668095367


In [9]:
# ML 5y cum2

print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_5y_cum2_226.xlsx")

0.15484713764739433
0.033425878616271144
1.909298975466369
0.46713033811840876


In [15]:
# ML 2022 cum1
print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_1y_2022_cum1_226.xlsx")

0.13219009210715216
0.06383528764228327
0.0
0.0


In [18]:
# ML 2022 cum2

print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_1y_2022_cum2_226.xlsx")

0.18055494368216288
0.07211856221947661
0.0
0.0


In [23]:
# PCA 5yr cum1
print(pairs_pnl_all["unrealized_pnl_cum"][-1])
print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_5y_cum1_226.xlsx")

-0.011350302362651998
0.0036979148283457068
0.00047642417213604937
2.513520274273743
0.009974286802673583


In [26]:
#PCA 5yr cum2 
print(pairs_pnl_all["unrealized_pnl_cum"][-1])
print(pairs_pnl_all["unrealized_PnL"].std())
print(pairs_pnl_all["realized_pnl_daily"].std())
print(pairs_pnl_all["realized_max_drawdown"].max())
print(pairs_pnl_all["realized_avg_drawdown"].max())


pairs_pnl_all.to_excel(path_output+"pairs_pnl_5y_cum2_226.xlsx")

0.0015097092167241088
0.002719535636502317
0.0006237786708593762
0.7036711908024562
0.002792345995247842
